# Урок 1

В этом уроке мы научимся извлекать MFCC признаки из аудио файла. Для этого необходимо сделать следующее:

1) Загрузить отсчеты и частоту дискретизации wav файла.<br>
2) Посчитать MFCC признаки по полученным выше данным.<br>
3) Записать посчитанные признаки в удобный формат.<br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa, librosa.display

from utils import FtrFile

In [ ]:
wav_example = 'data/example.wav'

# чтение wav файла:
x, sr = librosa.load(wav_example, sr=None)     # sr=None to preserve the native sampling rate

print('Number of samples: {}'.format(len(x)))
print('Sampling rate: {} Hz'.format(sr))
print('Duration: {:.2f} s'.format(len(x)/sr))  # ~ librosa.get_duration(x, sr)

# воспроизведение:
ipd.Audio(x, rate=sr)

Теперь представим данную запись во временной области.

In [ ]:
# амплитудная огибающая:

%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 5.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

librosa.display.waveplot(x, sr=sr)

In [ ]:
librosa.display.waveplot(x[10000:11000], sr=sr)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x[10000:11000])
plt.show()

### почему графики отличаются? 

In [ ]:
# спектрограмма:

D = librosa.amplitude_to_db(np.abs(librosa.stft(x)), ref=np.max)
librosa.display.specshow(D, y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')

### FBANK

In [ ]:
# n_fft = 25 ms      -- длина кадра
# hop_length = 10 ms -- длина шага
# тогда в отчетах получаем следующее:

n_fft = int(sr * 0.025)
hop_length = int(sr * 0.01)


fbanks = librosa.feature.melspectrogram(x, sr=sr, n_mels=40, n_fft=n_fft, hop_length=hop_length)
print(fbanks.shape)
print(fbanks[:,1])

### MFCC

In [ ]:
mfccs = librosa.feature.mfcc(S=librosa.power_to_db(fbanks), n_mfcc=13)
print(mfccs.shape)
print(mfccs[:,1])

In [ ]:
wav_name = wav_example.split('.')[0]
np.save(wav_name, mfccs)

В результате мы получили бинарный файл example.npy, содержащий в себе матрицу с 13-мерными MFCC признаками аудио файла example.wav

Но работать с бинарными файлами не всегда удобно. Мы будем пользоваться общепринятым ark текстовым форматом библиотеки распознавания речи KALDI, где в одном файле будут храниться признаки для всех используемых записей. Он выглядит следующим образом:

file1-identifier [<br>
vec0_0 vec0_1 vec0_2 ... vec0_12<br>
vec1_0 vec1_1 vec1_2 ... vec1_12<br>
....<br>
vecN_0 vecN_1 vecN_2 ... vecN_12<br>
]<br>
file2-identifier [<br>
vec0_0 vec0_1 vec0_2 ... vec0_12<br>
vec1_0 vec1_1 vec1_2 ... vec1_12<br>
....<br>
vecN_0 vecN_1 vecN_2 ... vecN_12<br>
]<br>

Здесь "file1-identifier" – это имя файла, а "vec0_0 vec0_1 vec0_2 ... vec0_12" – вектор MFCC признаков первого (нулевого) кадра записи. 
<br><b>Задание 1</b>.Сохранить нашу матрицу с признаками (features) в указанном формате: 

In [ ]:
# запись признаков в формат ark,t:
file_name = wav_name + '.txtftr'
features = mfccs

with open(file_name, 'w') as fn:
    #---------------------------TODO-----------------------------------------

    #------------------------------------------------------------------------    

Для чтения признаков из такого формата мы будем использовать удобную читалку FtrFile. Функция FtrFile.FtrDirectoryReader принимает файл с признаками и возвращает имя файла и класс features, который имеет следующие атрибуты:

- nSamples – количество векторов в фале;<br>
- nDim – размерность вектора признаков;<br>
- readvec() – возвращает очередной вектор признаков;<br>
- getall() – возвращает все признаки текущего файла.<br>

Также, для ее работы необходимо добавлять префикс 'ark,t:' к названию файла с признаками.

Попробуем с ее помощью прочитать наш example.txtftr:

In [ ]:
ark_file = 'ark,t:' + file_name
for fileName, features in FtrFile.FtrDirectoryReader(ark_file):
    print('имя файла: {}'.format(fileName))
    print('количество векторов в файле: {}'.format(features.nSamples))
    print('размерность вектора признаков: {}'.format(features.nDim))
    print('вектор признаков первого кадра: {}'.format(features.readvec()))

Для работы с признаками удобно делать цикл по количеству векторов в файле и в нем уже вызывать features.readvec() для получения признаков текущего кадра. Этим мы займемся в следующем уроке.